In [1]:
import numpy as np 
import numba as nb
import matplotlib as mp 
import matplotlib.pyplot as plt 
import cython as c 
from Cython.Build import cythonize
%config InlineBackend.figure_format = 'pdf'
%load_ext Cython

Attempt to plot: 
$$\sum_{i=1}^{n=\infty} \frac{\cos(3^i \pi x)}{2^i}$$

In [2]:
def func(X, n):
    i = 0
    A = np.zeros_like(X)
    for i in range(n):
        A += np.asarray(np.cos(3**i * np.pi * X) / (2**i))
    return A

@nb.jit(nopython=True)
def n_func(X, n):
    i = 0
    A = np.zeros_like(X)
    for i in range(n):
        A += np.asarray(np.cos(3**i * np.pi * X) / (2**i))
    return A

In [3]:
%%cython -a -c=-ffast-math
cimport cython

import numpy as np
cimport numpy as np
from libc.math cimport cos, pi

@cython.boundscheck(False)
@cython.wraparound(False)
def c_func(np.ndarray[double] X, int n):
    cdef int i = 0
#     cdef int N = X.shape[0]
#     cdef int M = X.shape[1]
    cdef np.ndarray[double] A = np.zeros_like(X, dtype=float)
    for i in range(n):
        A += np.asarray(np.cos(3**i * pi * X) / (2**i))
    return A

In [4]:
#%%time
x = np.linspace(0, np.pi/3, 10**5)
%timeit c_F = c_func(x, 64)
%timeit n_F = n_func(x, 64)
%timeit F = func(x, 64)
c_F = c_func(x, 64)
# print(c_F)

86.2 ms ± 5.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
213 ms ± 9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
102 ms ± 7.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
plt.figure(figsize=(16, 6))
plt.plot(x, c_F, linewidth=0.01)
plt.grid()
plt.show()

<Figure size 1152x432 with 1 Axes>